In [1]:
!ls drive/MyDrive/Classes/ece542/competition-project/

 2_conv-plus-normalization.ipynb
 attempt-1.ipynb
 Best
 best-conv-plus-normalization.ipynb
 Best_Models
'Copy of attempt-1.ipynb'
'Copy of conv-plus-normalization.ipynb'
'Copy of current-conv-plus-normalization.ipynb'
'Copy of Hyoer attempt-1.ipynb'
 current-conv-plus-normalization.ipynb
 data.zip
 experiments
'Hyoer attempt-1.ipynb'
 lstm
 ml_utils
 Prediction
 test_data.zip
 window3


In [2]:
!cp drive/MyDrive/Classes/ece542/competition-project/data.zip .
!cp drive/MyDrive/Classes/ece542/competition-project/window3/window3_data.zip .

In [3]:
!unzip data.zip
!unzip window3_data.zip

Archive:  data.zip
   creating: data/
   creating: data/preprocessed_data/
   creating: data/splits/
   creating: data/splits/test/
  inflating: data/splits/test/subject_007_session_02__x.csv  
  inflating: data/splits/test/subject_007_session_03__x.csv  
  inflating: data/splits/test/subject_005_session_03__x.csv  
  inflating: data/splits/test/subject_007_session_01__x.csv  
  inflating: data/splits/test/subject_006_session_01__y.csv  
  inflating: data/splits/test/subject_006_session_03__y.csv  
  inflating: data/splits/test/subject_008_session_01__y.csv  
  inflating: data/splits/test/subject_006_session_01__x.csv  
  inflating: data/splits/test/subject_006_session_02__y.csv  
  inflating: data/splits/test/subject_006_session_02__x.csv  
  inflating: data/splits/test/subject_007_session_02__y.csv  
  inflating: data/splits/test/subject_007_session_04__x.csv  
  inflating: data/splits/test/subject_007_session_01__y.csv  
  inflating: data/splits/test/subject_006_session_03__x.csv  


In [4]:
!rm data.zip
!rm window3_data.zip

In [5]:
# Access token redacted. Replace <ACCESS_TOKEN> with your access token.
!git clone https://savadikarc:<ACCESS_TOKEN>@github.com/savadikarc/ece542-competition-project.git

Cloning into 'ece542-competition-project'...
remote: Enumerating objects: 238, done.
remote: Counting objects: 100% (238/238), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 238 (delta 128), reused 158 (delta 60), pack-reused 0
Receiving objects: 100% (238/238), 275.09 KiB | 3.72 MiB/s, done.
Resolving deltas: 100% (128/128), done.


In [6]:
!cd ece542-competition-project && git checkout phase-1 && git pull origin phase-1

Already on 'phase-1'
Your branch is up to date with 'origin/phase-1'.
From https://github.com/savadikarc/ece542-competition-project
 * branch            phase-1    -> FETCH_HEAD
Already up to date.


In [1]:
base_path = "drive/MyDrive/Classes/ece542/competition-project"

In [2]:
import os
import sys
sys.path.append("ece542-competition-project")

In [3]:
import json
from collections import Counter
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
from torch.utils.data.sampler import WeightedRandomSampler
import numpy as np
from ml_utils.dataset import SubjectDataset

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cpu


In [5]:
base_data_path_1sec = os.path.join("data", "splits")
val_data_path_1sec = os.path.join(base_data_path_1sec, "val")
test_data_path_1sec = os.path.join(base_data_path_1sec, "test")

base_data_path_3sec = os.path.join("window3_splits")
val_data_path_3sec = os.path.join(base_data_path_3sec, "val")
test_data_path_3sec = os.path.join(base_data_path_3sec, "test")

splits_file = os.path.join(base_path, "experiments", "metadata", "split_ids.json")
with open(splits_file, "r") as f:
    split_ids = json.load(f)

In [6]:
stats_path = os.path.join(base_path, "experiments", "metadata", "statistics.json")
with open(stats_path, "r") as f:
    stats = json.load(f)

In [7]:
min = np.array([v["min"] for k, v in stats.items()])
max = np.array([v["max"] for k, v in stats.items()])

min = torch.from_numpy(min).float()
min = torch.unsqueeze((torch.unsqueeze(min, 0)), -1)
min = min.to(device)
max = torch.from_numpy(max).float().to(device)
max = torch.unsqueeze((torch.unsqueeze(max, 0)), -1)
max = max.to(device)

In [8]:
min.shape

torch.Size([1, 6, 1])

In [9]:
# Training hyperparameters
batch_size = 128

In [10]:
# Model definition
class OneDConvNet1(nn.Module):
  def __init__(self, n_features, n_classes, base_filters=32):
    super(OneDConvNet1, self).__init__()

    self.conv1 = nn.Conv1d(in_channels=n_features, out_channels=base_filters, kernel_size=3, stride=1, padding=1)
    self.norm1 = nn.LayerNorm(base_filters*40)
    self.pool1 = nn.AvgPool1d(kernel_size=2, stride=2)
    self.conv2 = nn.Conv1d(in_channels=base_filters, out_channels=base_filters*2, kernel_size=3, stride=1, padding=1)
    self.norm2 = nn.LayerNorm(base_filters*2*20)
    self.pool2 = nn.AvgPool1d(kernel_size=2, stride=2)
    self.conv3 = nn.Conv1d(in_channels=base_filters*2, out_channels=base_filters*4, kernel_size=3, stride=1, padding=1)
    self.norm3 = nn.LayerNorm(base_filters*4*10)
    self.pool3 = nn.AvgPool1d(kernel_size=2, stride=2)
    self.conv4 = nn.Conv1d(in_channels=base_filters*4, out_channels=base_filters*8, kernel_size=3, stride=1, padding=1)
    self.norm4 = nn.LayerNorm(base_filters*8*5)
    self.fc1 = nn.Linear(base_filters*8, base_filters*16)
    self.dropout5 = nn.Dropout(0.4)
    self.fc2 = nn.Linear(base_filters*16, n_classes)
  
  def forward(self, x):
    x = self.conv1(x)
    N, C, T = x.shape
    x = x.view(N, C*T)
    x = self.norm1(x)
    x = x.view(N, C, T)
    x = self.pool1(F.relu(x))

    x = self.conv2(x)
    N, C, T = x.shape
    x = x.view(N, C*T)
    x = self.norm2(x)
    x = x.view(N, C, T)
    x = self.pool2(F.relu(x))

    x = self.conv3(x)
    N, C, T = x.shape
    x = x.view(N, C*T)
    x = self.norm3(x)
    x = x.view(N, C, T)
    x = self.pool3(F.relu(x))

    x = self.conv4(x)
    N, C, T = x.shape
    x = x.view(N, C*T)
    x = self.norm4(x)
    x = x.view(N, C, T)
    x = F.relu(x)

    N, C, T = x.size()
    x = x.mean(dim=-1) # Flatten
    x = self.dropout5(F.relu(self.fc1(x)))
    x = self.fc2(x)
    return x

model1 = OneDConvNet1(6, 4).to(device)
model1.load_state_dict(torch.load(os.path.join(base_path, "experiments", "models", "checkpoint_instance_norm_dropout_sgd_lr3.pth")))
model1 = model1.to(device)

In [11]:
# Model definition
class OneDConvNet2(nn.Module):
  def __init__(self, n_features, n_classes):
    super(OneDConvNet2, self).__init__()
    self.conv1 = nn.Conv1d(in_channels=n_features, out_channels=16, kernel_size=3, stride=1, padding=1)
    self.norm1 = nn.LayerNorm(16*40)
    self.pool1 = nn.AvgPool1d(kernel_size=2, stride=2)
    self.conv2 = nn.Conv1d(in_channels= 16, out_channels=32, kernel_size=3, stride=1, padding=1)
    self.norm2 = nn.LayerNorm(32*20)
    self.pool2 = nn.AvgPool1d(kernel_size=2, stride=2)
    self.conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
    self.norm3 = nn.LayerNorm(64*10)
    self.pool3 = nn.AvgPool1d(kernel_size=2, stride=2)
    self.conv4 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
    self.norm4 = nn.LayerNorm(128*5)
    self.fc1 = nn.Linear(128, 64)
    self.dropout5 = nn.Dropout(0.2)
    self.fc2 = nn.Linear(64, n_classes)
  
  def forward(self, x):
    x = self.conv1(x)
    N, C, T = x.shape
    x = x.view(N, C*T)
    x = self.norm1(x)
    x = x.view(N, C, T)
    x = self.pool1(F.relu(x))

    x = self.conv2(x)
    N, C, T = x.shape
    x = x.view(N, C*T)
    x = self.norm2(x)
    x = x.view(N, C, T)
    x = self.pool2(F.relu(x))

    x = self.conv3(x)
    N, C, T = x.shape
    x = x.view(N, C*T)
    x = self.norm3(x)
    x = x.view(N, C, T)
    x = self.pool3(F.relu(x))

    x = self.conv4(x)
    N, C, T = x.shape
    x = x.view(N, C*T)
    x = self.norm4(x)
    x = x.view(N, C, T)
    x = F.relu(x)

    N, C, T = x.size()
    x = x.mean(dim=-1) # Flatten
    x = self.dropout5(F.relu(self.fc1(x)))
    x = self.fc2(x)
    return x

model2 = OneDConvNet2(6, 4).to(device)
model2.load_state_dict(torch.load(os.path.join(base_path, "experiments", "models", "checkpoint_norm_dropout_32_1_long.pth")))
model2 = model2.to(device)

In [12]:
class OneDConvNet3(nn.Module):
  def __init__(self, n_features, n_classes, base_filters=32):
    super(OneDConvNet3, self).__init__()

    self.conv1 = nn.Conv1d(in_channels=n_features, out_channels=base_filters, kernel_size=3, stride=1, padding=1)
    self.norm1 = nn.LayerNorm(base_filters*120)
    self.pool1 = nn.AvgPool1d(kernel_size=2, stride=2)
    self.conv2 = nn.Conv1d(in_channels=base_filters, out_channels=base_filters*2, kernel_size=3, stride=1, padding=1)
    self.norm2 = nn.LayerNorm(base_filters*2*60)
    self.pool2 = nn.AvgPool1d(kernel_size=2, stride=2)
    self.conv3 = nn.Conv1d(in_channels=base_filters*2, out_channels=base_filters*4, kernel_size=3, stride=1, padding=1)
    self.norm3 = nn.LayerNorm(base_filters*4*30)
    self.pool3 = nn.AvgPool1d(kernel_size=2, stride=2)
    self.conv4 = nn.Conv1d(in_channels=base_filters*4, out_channels=base_filters*8, kernel_size=3, stride=1, padding=1)
    self.norm4 = nn.LayerNorm(base_filters*8*15)
    self.fc1 = nn.Linear(base_filters*8*15, base_filters*16)
    self.dropout5 = nn.Dropout(0.5)
    self.fc2 = nn.Linear(base_filters*16, n_classes)
  
  def forward(self, x):
    x = self.conv1(x)
    N, C, T = x.shape
    x = x.view(N, C*T)
    x = self.norm1(x)
    x = x.view(N, C, T)
    x = self.pool1(F.relu(x))

    x = self.conv2(x)
    N, C, T = x.shape
    x = x.view(N, C*T)
    x = self.norm2(x)
    x = x.view(N, C, T)
    x = self.pool2(F.relu(x))

    x = self.conv3(x)
    N, C, T = x.shape
    x = x.view(N, C*T)
    x = self.norm3(x)
    x = x.view(N, C, T)
    x = self.pool3(F.relu(x))

    x = self.conv4(x)
    N, C, T = x.shape
    x = x.view(N, C*T)
    x = self.norm4(x)
    x = x.view(N, C, T)
    x = F.relu(x)

    N, C, T = x.size()
    x = x.view(-1, C*T) # Flatten
    x = self.dropout5(F.relu(self.fc1(x)))
    x = self.fc2(x)
    return x

base_filters = 32
interval = 3
checkpoint_save_path = os.path.join(base_path, "experiments", "models", "window3")
filename = f"cnn_{base_filters}_interval{interval}.pth"
filename_long = f"cnn_{base_filters}_interval{interval}_long.pth"
model3 = OneDConvNet3(6, 4, base_filters).to(device)
model3.load_state_dict(torch.load(os.path.join(checkpoint_save_path, filename)))
model3.to(device)

OneDConvNet3(
  (conv1): Conv1d(6, 32, kernel_size=(3,), stride=(1,), padding=(1,))
  (norm1): LayerNorm((3840,), eps=1e-05, elementwise_affine=True)
  (pool1): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
  (conv2): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (norm2): LayerNorm((3840,), eps=1e-05, elementwise_affine=True)
  (pool2): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
  (conv3): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (norm3): LayerNorm((3840,), eps=1e-05, elementwise_affine=True)
  (pool3): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
  (conv4): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=(1,))
  (norm4): LayerNorm((3840,), eps=1e-05, elementwise_affine=True)
  (fc1): Linear(in_features=3840, out_features=512, bias=True)
  (dropout5): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=512, out_features=4, bias=True)
)

In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

class Evaluation:

  def __init__(self):
        pass
  
  def aggregate_metrics(self, y, y_hat):
    
    """ Metrics for entire set 
        
        Arguments passed are: 
                      y     : ground truth  
                      y_hat : predicted class
        Calculations:
                  accuracy  : (tp + tn) / (total samples =(p + n))
                  precision : tp / (tp + fp)
                  recall    : tp / (tp + fn)
                  f1        : 2 tp / (2 tp + fp + fn)
    """
    accuracy  = accuracy_score(y, y_hat)
    precision = precision_score(y, y_hat, average="macro")
    recall    = recall_score(y, y_hat, average="macro")
    f1        = f1_score(y, y_hat, average="macro")
    return {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1": f1
        }

  def classwise_metrics(self, y, y_hat):

    """ Metrics for each class. 
        Average is set to macro for calculating the score of each label, and find their unweighted mean. 
        This does not take label imbalance into account.
        
        Arguments passed are: 
                      y     : ground truth  
                      y_hat : predicted class
        Calculations:
                  accuracy  : (tp + tn) / (total samples =(p + n))
                  precision : tp / (tp + fp)
                  recall    : tp / (tp + fn)
                  f1        : 2 tp / (2 tp + fp + fn)
    """
    
    class_precision = precision_score(y, y_hat, average=None)
    class_recall    = recall_score(y, y_hat, average=None)
    class_f1        = f1_score(y, y_hat, average=None)
    
    return {
        "Precision": class_precision,
        "Recall": class_recall,
        "F1": class_f1
    }

In [14]:
evaluator = Evaluation()

In [15]:
val_dataset_1sec = SubjectDataset(
    val_data_path_1sec, 
    split_ids["val"]
)
val_dataloader_1sec = DataLoader(val_dataset_1sec, batch_size=batch_size, shuffle=False)
val_iterations_1sec = (len(val_dataset_1sec) // batch_size) + ((len(val_dataset_1sec) % batch_size) != 0)

val_dataset_3sec = SubjectDataset(
    val_data_path_3sec, 
    split_ids["val"]
)
val_dataloader_3sec = DataLoader(val_dataset_3sec, batch_size=batch_size, shuffle=False)
val_iterations_3sec = (len(val_dataset_3sec) // batch_size) + ((len(val_dataset_3sec) % batch_size) != 0)

Converting uid 001_08
Converting uid 002_01
Converting uid 001_01
Converting uid 001_04
Converting uid 001_08
Converting uid 002_01
Converting uid 001_01
Converting uid 001_04


In [16]:
output = []
labels = []
for (X_1sec, y_1sec), (X_3sec, y_3sec) in zip(val_dataloader_1sec, val_dataloader_3sec):
    X_1sec = X_1sec.float().to(device)
    X_1sec = (X_1sec - min) / (max - min)

    X_3sec = X_3sec.float().to(device)
    X_3sec = (X_3sec - min) / (max - min)

    y = y_1sec.view(X_1sec.size(0)).to(device)
    y_3sec = y_3sec.view(X_3sec.size(0)).to(device)
    assert torch.sum(y == y_3sec).item() == y.shape[0]

    y_pred1 = model1(X_1sec)
    y_pred2 = model2(X_1sec)
    y_pred3 = model3(X_3sec)

    y_pred1 = F.softmax(y_pred1, dim=1)
    y_pred2 = F.softmax(y_pred2, dim=1)
    y_pred3 = F.softmax(y_pred3, dim=1)

    y_pred = (y_pred1 + y_pred2 + y_pred3)/3.
    predicted_classes = torch.argmax(y_pred, dim=1).detach().cpu().numpy()
    y_true = y.cpu().numpy()

    output.append(predicted_classes)
    labels.append(y_true)

_output = np.concatenate(output, axis=0)
_labels = np.concatenate(labels, axis=0)    

In [17]:
agg_metrics = evaluator.aggregate_metrics(_labels, _output)
print("Agg metrics")
print(agg_metrics)
classwise_metrics = evaluator.classwise_metrics(_labels, _output)
print("Classwise Metrics")
print(classwise_metrics)

Agg metrics
{'Accuracy': 0.8189711147802534, 'Precision': 0.7810045191734295, 'Recall': 0.8020999994282187, 'F1': 0.7809772682251673}
Classwise Metrics
{'Precision': array([0.83934186, 0.84309357, 0.63398533, 0.80759732]), 'Recall': array([0.90252423, 0.8982706 , 0.85492911, 0.55267606]), 'F1': array([0.86978714, 0.86980791, 0.72806402, 0.65625   ])}


In [19]:
test_dataset_1sec = SubjectDataset(
    test_data_path_1sec, 
    split_ids["test"]
)
test_dataloader_1sec = DataLoader(test_dataset_1sec, batch_size=batch_size, shuffle=False)
test_iterations_1sec = (len(test_dataset_1sec) // batch_size) + ((len(test_dataset_1sec) % batch_size) != 0)

test_dataset_3sec = SubjectDataset(
    test_data_path_3sec, 
    split_ids["test"]
)
test_dataloader_3sec = DataLoader(test_dataset_3sec, batch_size=batch_size, shuffle=False)
test_iterations_3sec = (len(test_dataset_3sec) // batch_size) + ((len(test_dataset_3sec) % batch_size) != 0)

Converting uid 005_03
Converting uid 006_01
Converting uid 006_02
Converting uid 006_03
Converting uid 007_01
Converting uid 007_02
Converting uid 007_03
Converting uid 007_04
Converting uid 008_01
Converting uid 005_03
Converting uid 006_01
Converting uid 006_02
Converting uid 006_03
Converting uid 007_01
Converting uid 007_02
Converting uid 007_03
Converting uid 007_04
Converting uid 008_01


In [22]:
output = []
labels = []
for (X_1sec, y_1sec), (X_3sec, y_3sec) in zip(test_dataloader_1sec, test_dataloader_3sec):
    X_1sec = X_1sec.float().to(device)
    X_1sec = (X_1sec - min) / (max - min)

    X_3sec = X_3sec.float().to(device)
    X_3sec = (X_3sec - min) / (max - min)

    y = y_1sec.view(X_1sec.size(0)).to(device)
    y_3sec = y_3sec.view(X_3sec.size(0)).to(device)
    assert torch.sum(y == y_3sec).item() == y.shape[0]

    y_pred1 = model1(X_1sec)
    y_pred2 = model2(X_1sec)
    y_pred3 = model3(X_3sec)

    y_pred1 = F.softmax(y_pred1, dim=1)
    y_pred2 = F.softmax(y_pred2, dim=1)
    y_pred3 = F.softmax(y_pred3, dim=1)

    y_pred = (y_pred1 + y_pred2 + y_pred3)/3.
    predicted_classes = torch.argmax(y_pred, dim=1).detach().cpu().numpy()
    y_true = y.cpu().numpy()

    output.append(predicted_classes)
    labels.append(y_true)

_output = np.concatenate(output, axis=0)
_labels = np.concatenate(labels, axis=0)

In [23]:
agg_metrics = evaluator.aggregate_metrics(_labels, _output)
print("Agg metrics")
print(agg_metrics)
classwise_metrics = evaluator.classwise_metrics(_labels, _output)
print("Classwise Metrics")
print(classwise_metrics)

Agg metrics
{'Accuracy': 0.8973454841088276, 'Precision': 0.8442889789932634, 'Recall': 0.9295998169478218, 'F1': 0.8798249370840083}
Classwise Metrics
{'Precision': array([0.98039807, 0.83204799, 0.91117534, 0.65353452]), 'Recall': array([0.8871236 , 0.93756035, 0.97868377, 0.91503154]), 'F1': array([0.93143151, 0.8816586 , 0.94372381, 0.76248583])}
